In [1]:
import os

import pandas as pd
import numpy as np


from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate
)

# Import Azure OpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

from dotenv import load_dotenv

from collections import Counter

from reclaim_tiktok.transcriber.db_connector import DBConnector

%load_ext autoreload
%autoreload 2

In [4]:
# Construct an absolute path to where your .env file is located
dotenv_path = os.path.join(os.getcwd(), '.env')
result = load_dotenv(dotenv_path)
print(result)  # This should print True if the file is loaded successfully

False


In [5]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small-eastus",
    openai_api_version="2024-02-01",
)

llm_client = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="gpt-35-turbo-16k",
)

## Get data

In [7]:
db_connector = DBConnector()

In [18]:
rows = db_connector.get_videos_with_german_transcription_without_core_message()

In [19]:
len(rows)

23

In [10]:
print(rows[0])

(7240048311242067227, datetime.datetime(2023, 6, 2, 13, 25, 16), datetime.datetime(2024, 5, 16, 13, 48, 18, 577000), 62, 1057, 102, 73, 10700, 'Was ist los mit unseren Sozialkassen? ', False, 7003316956967666693, 'None', 'https://www.tiktok.com/@joana_cotar/video/7240048311242067227', None, 'Ich lese gerade das payonia hauptstadtbriefing. Das bekomme ich jeden Morgen und darin ist heute zu lesen, nächstes Jahr droht schon wieder 1 milliardendefizit in der gesetzlichen Krankenversicherung. Es fehlen 7 Milliarden Euro, die irgendwo herkommen müssen. Und im dazugehörigen Artikel schreibt der Pionier, teuer teurer am teuersten, was ist los mit unseren sozialkassen? Und ich kann Ihnen sagen, was los ist mit unseren sozialkassen. Wenn sie Millionen von Menschen direkt in unsere Sozialsysteme einwandern lassen und sie direkt davon profitieren, ohne auch jemals so einen Cent dafür eingezahlt zu haben, dann sind die Kassen eben irgendwann leer. Dann müssen die woanders aufgefüllt werden. Und da

## Produce core messages for transcripts longer than 800

In [11]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "Du bist ein Experte im Schreiben. Du hilfst mir, die Kernaussagen aus den Transcripts von TikTokVideos zu ziehen."
            )
        ),
        HumanMessagePromptTemplate.from_template("Ziehe aus dem folgenden deutschen Transkript von einem Tiktok-Videos eine möglichst kurze Beschreibung bestehend aus maximal drei und nur maximal drei kurzen und prägnanten Kernaussagen, ohne Titel, starte direkt mit den Kernaussagen. Maximal 8 Wörter pro Kernaussage: \n\nPOSTS:{posts}\n\nTOPIC TITLE:"),
    ]
)

In [12]:
rows[0][14]

'Ich lese gerade das payonia hauptstadtbriefing. Das bekomme ich jeden Morgen und darin ist heute zu lesen, nächstes Jahr droht schon wieder 1 milliardendefizit in der gesetzlichen Krankenversicherung. Es fehlen 7 Milliarden Euro, die irgendwo herkommen müssen. Und im dazugehörigen Artikel schreibt der Pionier, teuer teurer am teuersten, was ist los mit unseren sozialkassen? Und ich kann Ihnen sagen, was los ist mit unseren sozialkassen. Wenn sie Millionen von Menschen direkt in unsere Sozialsysteme einwandern lassen und sie direkt davon profitieren, ohne auch jemals so einen Cent dafür eingezahlt zu haben, dann sind die Kassen eben irgendwann leer. Dann müssen die woanders aufgefüllt werden. Und dann dürfen diejenigen, die dieses Land eh schon am laufen halten, halt noch tiefer in die Tasche greifen, damit die Politiker in Berlin noch mehr Geld zum umverteilen haben. Man könnte allerdings auch beim bussprungsproblem ansetzen, aber daran hat die Ampel kein Interesse. '

In [13]:
import re

In [ ]:
# produce core message
i=0
len_rows=len(rows)
for row in rows:
    try:
        print(f"{i}/{len_rows}\r", end="", flush=True)
        transcript=row[14]
        if len(transcript)>800:
            prompt = chat_template.format_messages(posts=transcript)
            core_message=llm_client.invoke(prompt).content
            # Remove specific substrings using regular expressions
            core_message = re.sub(r'(Kernaussage|n:|1\.|2\.|3\.)', '', core_message)
            # Replace newline characters with semicolons
            result = re.sub(r'\n', ';', core_message)
            # add results to core_messages_de column in db by video_id
        else:
            # add transcript_de row[14] directly as core_messages_de
            result = transcript

        db_connector.update_core_messages(video_id=row[0], core_messages_de=result)
        i+=1
    except Exception as e:
        print(f"Error processing index {row[0]}: {e}")
        continue